In [7]:
from dataset import cargarDataset
from pathlib import Path
import time
from ej2 import pinvEcuacionesNormales
from ej3 import pinvSVD
from ej4 import pinvHouseHolder, pinvGramSchmidt
from ej5 import esPseudoInversa
from ej6 import validate_transferlearning, matriz_confusion
from alc import svd_reducida, QR_con_GS, transpuesta, QR_con_HH

In [2]:
data_path = Path("./dataset/cats_and_dogs")
X_train, Y_train, X_val, Y_val = cargarDataset(data_path)

In [5]:
    #Cholesky
    print("running cholesky...")
    start_time = time.perf_counter()
    W = pinvEcuacionesNormales(X_train, transpuesta(Y_train))
    end_time = time.perf_counter()
    Cholesky_time = end_time - start_time
    print(f"Cholesky exercise executed in: {Cholesky_time:.4f} seconds")
    Cholesky_accuracy = validate_transferlearning(W,X_val,Y_val)
    print("W shape", W.shape)
    print("X_val shape", X_val.shape)
    print("Y_val shape", Y_val.shape)
    matriz_confusion(W, X_val, Y_val)

running cholesky...
Rango de X: 1536
Cholesky exercise executed in: 42.7910 seconds

--- Resultados de Validación ---
Precisión (Accuracy): 68.40%
Clasificó correctamente 684 de 1000 muestras.
W shape (2, 1536)
X_val shape (1536, 1000)
Y_val shape (1000, 2)

   --- Matriz de Confusión (Validación) ---  
               Predicción: GATO | Predicción: PERRO |
                 -------------------------------------
Realidad: GATO  |       334       |        166        |
                 -------------------------------------
Realidad: PERRO |       150       |        350        |
                 -------------------------------------




In [4]:
    #SVD
    print("running svd...")
    start_time = time.perf_counter()
    U, S, V = svd_reducida(X_train)
    W_SVD = pinvSVD(U, S, V, Y_train)
    end_time = time.perf_counter()
    SVD_time = end_time - start_time
    print(f"SVD exercise executed in: {SVD_time:.4f} seconds")
    SVD_accuracy = validate_transferlearning(W_SVD,X_val,Y_val)
    matriz_confusion(W_SVD, X_val, Y_val)


running svd...
Calculando A*AT...
A*AT calculada en 19.6211 segundos
Calculando DiagRH de A*AT para obtener diagonalización
Calculando metpot 0/1536 con dimension (1536, 1536)...
Calculando metpot 100/1536 con dimension (1436, 1436)...
Calculando metpot 200/1536 con dimension (1336, 1336)...
Calculando metpot 300/1536 con dimension (1236, 1236)...
Calculando metpot 400/1536 con dimension (1136, 1136)...
El método no convergió después de 1000 iteraciones.
Calculando metpot 500/1536 con dimension (1036, 1036)...
Calculando metpot 600/1536 con dimension (936, 936)...
Calculando metpot 700/1536 con dimension (836, 836)...
El método no convergió después de 1000 iteraciones.
Calculando metpot 800/1536 con dimension (736, 736)...
Calculando metpot 900/1536 con dimension (636, 636)...
Calculando metpot 1000/1536 con dimension (536, 536)...
Calculando metpot 1100/1536 con dimension (436, 436)...
El método no convergió después de 1000 iteraciones.
Calculando metpot 1200/1536 con dimension (336, 

In [ ]:
    #QR
    print("running qr gs...")
    start_time = time.perf_counter()
    Y_train_T = transpuesta(Y_train)
    X_train_T = transpuesta(X_train)
    Q, R = QR_con_GS(X_train_T)
    W_GS =  pinvGramSchmidt(Q, R, Y_train_T)
    print("W_GS shape", W_GS.shape)
    print("X_val shape", X_val.shape)
    print("Y_val shape", Y_val.shape)

    end_time = time.perf_counter()
    GS_time = end_time - start_time
    print(f"GS exercise executed in: {GS_time:.4f} seconds")
    GS_accuracy = validate_transferlearning(W_GS,X_val,Y_val)
    matriz_confusion(W_GS, X_val, Y_val)

running qr gs...
W_GS shape (2, 1536)
X_val shape (1536, 1000)
Y_val shape (1000, 2)
GS exercise executed in: 1764.1974 seconds

--- Resultados de Validación ---
Precisión (Accuracy): 68.40%
Clasificó correctamente 684 de 1000 muestras.

   --- Matriz de Confusión (Validación) ---  
               Predicción: GATO | Predicción: PERRO |
                 -------------------------------------
Realidad: GATO  |       334       |        166        |
                 -------------------------------------
Realidad: PERRO |       150       |        350        |
                 -------------------------------------




In [8]:
    #QR
    print("running qr hh...")
    start_time = time.perf_counter()
    Y_train_T = transpuesta(Y_train)
    X_train_T = transpuesta(X_train)
    Q, R = QR_con_HH(X_train_T)
    W_HH =  pinvHouseHolder(Q, R, Y_train_T)
    print("W_HH shape", W_HH.shape)
    print("X_val shape", X_val.shape)
    print("Y_val shape", Y_val.shape)

    end_time = time.perf_counter()
    HH_time = end_time - start_time
    print(f"HH exercise executed in: {HH_time:.4f} seconds")
    HH_accuracy = validate_transferlearning(W_HH,X_val,Y_val)
    matriz_confusion(W_HH, X_val, Y_val)

running qr hh...
W_HH shape (2, 1536)
X_val shape (1536, 1000)
Y_val shape (1000, 2)
HH exercise executed in: 1200.0205 seconds

--- Resultados de Validación ---
Precisión (Accuracy): 68.40%
Clasificó correctamente 684 de 1000 muestras.

   --- Matriz de Confusión (Validación) ---  
               Predicción: GATO | Predicción: PERRO |
                 -------------------------------------
Realidad: GATO  |       334       |        166        |
                 -------------------------------------
Realidad: PERRO |       150       |        350        |
                 -------------------------------------




In [ ]:
    print("\n" + "--- Tablas de resultados ---".center(45))
    print(" " * 15 + " Tiempo | Accuracy |")
    print(" " * 17 + "-" * 37)
    print(f"SVD  | {SVD_time:^15} | {SVD_accuracy:^17} |")
    print(" " * 17 + "-" * 37)
    print(f"GS | {GS_time:^15} | {GS_accuracy:^17} |")
    print(" " * 17 + "-" * 37)
    print(f"CHL | {Cholesky_time:^15} | {Cholesky_accuracy:^17} |")
    print(" " * 17 + "-" * 37)
    print("\n")